# Sets

There are five kinds of nodes: forests , sawmills, and transshipment hubs. While forests and sawmills act as sources and sinks (respectively) in the real-world representation of the problem, we introduce artificial sinks and sources for each commodity to facilitate modelling. Additionally, the nodes for the hubs represent potential locations to build hubs. Therefore, our graph consists of:

\begin{align*}
G =& (N, A) \\
N =& N^F  && \text{ (forests)} \\
   & \cup N^S && \text{ (sawmills)} \\
   & \cup N^H && \text{ (potential location for hubs)} \\
   & \cup S && \text{ (artificial source)}\\ 
   & \cup T && \text{ (artificial sink)}
\end{align*}

There are two types of vehicles available:
\begin{align*}
V =&\; \{\text{Truck, Transporter}\}
\end{align*}

Next we define the arcs: according to the exercise, transporting logs between sawmills and between forests is prohibited. Transporting "backwards" (from sawmills to forests, sawmills to hubs, and hubs to forests) is not possible We further prohibit transporting between hubs. At last, we need arcs to connect the forests with the source and the sawmills with the sink. The set of viable arcs thus is:
\begin{align*}
    A = (N^F \times N^H) \cup (N^H \times N^S) \cup (N^F \times N^S) \cup (S \times N^F) \cup (T \times N^S)
\end{align*}

Up until now, the model is only specified for a single commodity. We have K different commodities, where $$K\in \mathbb{N}$$

In further notation, every $k$ will denote one commodity, such that $$k \in \{1, \dots, K\}$$.

We need to adapt the graph to the multiple commodities. In principle, we assume that each forest can produce every commodity and each sawmill accepts all commodities (we will specifiy demand and supply further down). However, each commodity has its own sink and source; thus, the set of nodes is updated with a different set of source and sinks:

\begin{align*}
S = \{S_1, \dots, S_k, \dots, S_K\} \\
T = \{T_1, \dots, T_k, \dots, T_K\}
\end{align*}

The set of arcs remains the same, since each sawmill is connected to every sink and each forest is connected to every source.

# Variables

There are two decision variables. The first is the number of units of a certain commodity transported through arc $(i,j)$:

$$x_{ij}^k \in \mathbb{N}_0$$

The second variable is the decision on whether to build a hub at a potential location.

\begin{align*}
y_i =
  \begin{cases}
    1 & \text{if hub } i \text{ is built}\\
    0 & \text{otherwise}.
  \end{cases} \quad  \forall i \in N^H
\end{align*}

# Parameters

## Node-based parameters

Some parameters are related to the nodes. Hubs have fixed operation costs and unit costs of handling logs.

\begin{align*}
o_i \in \mathbb{R}^{+} \quad | \quad \forall i \in N^H & \text{ (operation cost of hub i)}\\
h_{i}^k \in \mathbb{R}^{+} \quad | \quad \forall i \in N^H & \text{ (handling cost per unit of commodity k at hub i)}\\
\end{align*}

Both forests and sawmills have a maximum capacity for each commodity, which are supply $s_i^k$ (for forests) and demand $d_i^k$ (for sawmills) respectively.

\begin{align*}
s_i^k \in \mathbb{Z}^+ \quad | \quad \forall i \in N^F \\
d_i^k \in \mathbb{Z}^+ \quad | \quad \forall i \in N^S
\end{align*}

## Arc-based parameters

Every arc can be travelled by at most one of a specified type of vehicle. That is, all arcs that connect to forests can only be travelled by transporters; arcs between hubs and sawmills can only be travelled by transporters. Which vehicle is used on an arc $(i,j)$ is denoted by $v_{ij}$, so the capacity of an arc is:

$$\kappa_{ij} = \kappa_{v_{ij}} \quad | \quad \forall (i,j) \in A\setminus \{(S \times N^F) \cup (T \times N^S)\},\; v \in V$$

where $\kappa_{Truck} > \kappa_{Transporter}$.

The capacity of an arc that connects the source of a commodity to a forest is equal to the supply of that forest:

$$\kappa_{S^ki} = s_i \quad | \quad \forall i \in N^F, \forall k \in K$$

In similar fashion, the capacity of the arcs connecting sawmills with the sink for each resource is equal to the demand of this sawmill:

$$\kappa_{iT^k} = s_i \quad | \quad \forall i \in N^S, \forall k \in K$$

The total cost of transporting a unit of a commodity $k$ on an arc $(i,j)$ is $c_{ij}^k$ is the product of the vehicle-specific cost of transporting a commodity $k$ per unit distance and the 

\begin{align*}
c_{ij}^k \in \mathbb{Z}^+
\end{align*}

The total cost of transporting a commodity on an arc will be the cost per unit of commodity, multiplied by the number of units of that commodity transported (will be defined further down in the objective).

The transport costs from sources and to sinks are 0. Since the sources and sink nodes are artificial, they pose no real costs.

$$c_{S^kj}^k =  c_{iT^k}^k = 0, \; \forall i \in N^S, \; \forall j \in N^F,\;\forall k \in K $$


## Other parameters

Finally, the price $p^k$ of a commodity $k$ accrues whenever it is delivered to its sink $T_k$.

$$p^k \in \mathbb{R}^+$$

# Objective function

The first part of the objective function is the revenue generated by all the commodities.

$$
\sum_{k=1}^K \sum_{i \in N^s} x_{iT_k}^k p^k 
$$

Note that the revenue is only generated for units of commodity reaching the respective sink. However, the constraints will make sure that every unit of commodity that leaves a source will also reach the sink.

First off, there are handling costs that occur at each hub for handling a certain number of units of a commodity. Note that the number of units of each commodity handled is equal to the sum of the units of this commodity reaching the hub from the forests.

$$\sum_{k = 1}^{K} \Big[ \sum_{j \in N^H} \big[ \sum_{i \in N^F} x_{ij}^k h_j^k \big] \Big]$$

We also have to substract the costs that stem from operating hubs (the activation or operating costs). 

$$
\sum_{i \in N^H}o_i y_i
$$

Also, transporting the commodities also produces costs. The transporting cost occurs at every arc $(i,j) \in A$ and is dependent on the number of logs for each type $k$ transported.

$$\sum_{\forall (i,j) \in A} x_{ij}^kc_{ij}^k $$

The ultimate objective function is as follow:

\begin{align*}
 \max \bigg( & \sum_{k=1}^K \sum_{i \in N^s} x_{iT_k}^k p^k && \text{ (revenue) } \\
- &  \sum_{k = 1}^{K} \Big[ \sum_{j \in N^H} \big[ \sum_{i \in N^F} x_{ij}^k h_j^k \big] \Big] && \text{ (handling costs at hubs) } \\
- &  \sum_{i \in N^H}o_i y_i && \text{ (fixed operating costs for hubs)} \\
- &  \sum_{\forall (i,j) \in A} x_{ij}^kc_{ij}^k \bigg) && \text{ (transportation costs)} 
\end{align*}

# Constraints

1) Make sure that only the right kind of commodity can reach the sink / source.

That is, for any arc connecting a sawmill $N^S$ and a sink $T^k$, the number of logs of type $l \neq k$ flowing through that arc has to be 0.

\begin{align*}
x_{iT_k}^l = 0 \quad | \quad \forall i \in N^S, \; \forall k \in K, \; \forall l \in K \setminus \{k\}
\end{align*}


The same holds for arcs between the sources and the forests.

\begin{align*}
x_{S_ki}^l = 0 \quad | \quad \forall i \in N^F, \; \forall k \in K, \; \forall l \in K \setminus \{k\} 
\end{align*}

2) The total number of logs of all types transported on arc $(i,j)$ cannot exceed the arc capacity $\kappa_{ij}$

$$\sum_{k=1}^K x_{ij}^k \leq \kappa_{ij}$$

3) The capacity of arcs connecting the forests with the sources and the sawmills with the sinks has already been established as the supply / demand of the respective forest. We model the flow conservation and account for the possibility of the demand not matching the supply; therefore, we have to allow for the net inflow of any forest being smaller or equal than the supply of this forest (and similarly, for sawmills). However, we instead decide to have a flow conservation for every node except for sources and sinks. In all other nodes, the net inflow has to be 0. This way, any "residual" supply or demand (that is, supply or demand that has not been fully matched by the optimal solution of the model) will "remain" in the source or sink.

$$
\sum_{\forall i \in N^F}x_{S^ki}^k \leq d_i^k \quad \forall k \in K \\
\sum_{\forall i \in N^S}x_{iT^k}^k \leq s_i^k \quad \forall k \in K \\
\sum_{\forall (i,j) \in A}x_{ij}^k = \sum_{\forall (j,i) \in A}x_{ji}^k \quad \forall i \in N,\; k \in K
$$

Say that trying to make sure that the difference in the vehicle cost that comes into plays; derive arc cost from vehicle and arc type with a summation?